In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


 ![](image/2019-11-25-14-25-47.png)

 # 1. 事前に準備済みのK8s環境を確認

 ## 1-1. K8s

In [53]:
%%bash
az aks show -g rg_bdc -n o9o9aks

Name     Location    ResourceGroup    KubernetesVersion    ProvisioningState    Fqdn
-------  ----------  ---------------  -------------------  -------------------  -----------------------------------------------------
o9o9aks  japaneast   rg_bdc           1.14.6               Succeeded            o9o9aks-rgbdc-745830-7ebfd297.hcp.japaneast.azmk8s.io


In [54]:
%%bash
az aks browse -g rg_bdc --listen-address 0.0.0.0 -n o9o9aks


Process is interrupted.


 ## 1-2. Node

In [55]:
%%bash
kubectl get node -o wide

NAME                       STATUS   ROLES   AGE   VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
aks-nodepool1-15232104-0   Ready    agent   65d   v1.14.6   10.240.0.4    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6
aks-nodepool1-15232104-1   Ready    agent   65d   v1.14.6   10.240.0.6    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6
aks-nodepool1-15232104-2   Ready    agent   65d   v1.14.6   10.240.0.5    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6


In [56]:
%%bash
az vm list -g MC_RG_BDC_O9O9AKS_JAPANEAST -d

Name                      ResourceGroup                PowerState    PublicIps    Fqdns    Location    Zones
------------------------  ---------------------------  ------------  -----------  -------  ----------  -------
aks-nodepool1-15232104-0  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-1  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-2  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast


In [57]:
%%bash
kubectl describe node


Name:               aks-nodepool1-15232104-0
Roles:              agent
Labels:             agentpool=nodepool1
                    beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=Standard_B16ms
                    beta.kubernetes.io/os=linux
                    failure-domain.beta.kubernetes.io/region=japaneast
                    failure-domain.beta.kubernetes.io/zone=0
                    kubernetes.azure.com/cluster=MC_rg_bdc_o9o9aks_japaneast
                    kubernetes.azure.com/role=agent
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=aks-nodepool1-15232104-0
                    kubernetes.io/os=linux
                    kubernetes.io/role=agent
                    node-role.kubernetes.io/agent=
                    storageprofile=managed
                    storagetier=Premium_LRS
Annotations:        node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-det

 # 2. operator定義

In [58]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/1.operator.yaml


# apiVersion: v1
# kind: Namespace
# metadata: {name: mssql-ag}
---
apiVersion: v1
kind: ServiceAccount
metadata: {name: mssql-operator, namespace: mssql-ag}
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata: {name: mssql-operator-ag}
rules:
- apiGroups: ['']
  resources: [serviceaccounts, services]
  verbs: [create, get, update, delete]
- apiGroups: [batch]
  resources: [jobs]
  verbs: [create, get, update, delete]
- apiGroups: [rbac.authorization.k8s.io]
  resources: [roles, rolebindings]
  verbs: [create, get, update, delete]
- apiGroups: [apps]
  resources: [statefulsets]
  verbs: [create, delete, get, update]
- apiGroups: ['']
  resources: [configmaps, endpoints, secrets]
  verbs: [create, get, update, watch, delete]
- apiGroups: ['']
  resources: [pods]
  verbs: [get, list, update]
- apiGroups: [apiextensions.k8s.io]
  resources: [customresourcedefinitions]
  verbs: [create]
- apiGroups: [apiextensions.k8s.io]
  resourceNames: [sqlservers.mssql.microsoft.com

 # 3. pod定義

In [59]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/2.sqlserver.yaml


apiVersion: mssql.microsoft.com/v1
kind: SqlServer
metadata:
  labels: {name: mssql1, type: sqlservr}
  name: mssql1
  namespace: mssql-ag
spec:
  acceptEula: true
  agentsContainerImage: mcr.microsoft.com/mssql/ha:2019-CTP3.2-ubuntu
  availabilityGroups: [mssql-ag]
  instanceRootVolumeClaimTemplate:
    accessModes: [ReadWriteOnce]
    resources:
      requests: {storage: 5Gi}
    storageClass: default
  saPassword:
    secretKeyRef: {key: sapassword, name: sql-secrets}
  sqlServerContainer: {image: 'mcr.microsoft.com/mssql/server:2019-CTP3.2-ubuntu'}
---
apiVersion: v1
kind: Service
metadata: {name: mssql1, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433}
  selector: {name: mssql1, type: sqlservr}
  type: LoadBalancer
---
apiVersion: mssql.microsoft.com/v1
kind: SqlServer
metadata:
  labels: {name: mssql2, type: sqlservr}
  name: mssql2
  namespace: mssql-ag
spec:
  acceptEula: true
  agentsContainerImage: mcr.microsoft.com/mssql/ha:2019-CTP3.2-ubuntu
  availabilityGro

 # 4. Service定義

In [60]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/3.ag-services.yaml


apiVersion: v1
kind: Service
metadata: {annotations: null, name: ag-primary, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433, targetPort: 1433}
  selector: {role.ag.mssql.microsoft.com/mssql-ag: primary, type: sqlservr}
  type: LoadBalancer
---
apiVersion: v1
kind: Service
metadata: {annotations: null, name: ag-secondary, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433}
  selector: {role.ag.mssql.microsoft.com/mssql-ag: secondary,
    type: sqlservr}
  type: LoadBalancer
---

 # 5. 環境確認

In [61]:
%%bash
kubectl get pod -o wide -n mssql-ag

NAME                              READY   STATUS      RESTARTS   AGE   IP             NODE                       NOMINATED NODE   READINESS GATES
mssql-initialize-mssql1-2bzpt     0/1     Error       0          50m   10.244.2.93    aks-nodepool1-15232104-0   <none>           <none>
mssql-initialize-mssql1-vd8rn     0/1     Completed   0          34m   10.244.2.95    aks-nodepool1-15232104-0   <none>           <none>
mssql-initialize-mssql2-9f8cr     0/1     Error       0          50m   10.244.1.105   aks-nodepool1-15232104-2   <none>           <none>
mssql-initialize-mssql2-znqvb     0/1     Completed   0          34m   10.244.0.72    aks-nodepool1-15232104-1   <none>           <none>
mssql-initialize-mssql3-754hk     0/1     Completed   0          50m   10.244.0.70    aks-nodepool1-15232104-1   <none>           <none>
mssql-operator-65459979cf-7vkk4   1/1     Running     0          51m   10.244.0.69    aks-nodepool1-15232104-1   <none>           <none>
mssql1-0                        

In [62]:
%%bash
kubectl get pvc -o wide -n mssql-ag
# kubectl describe pvc -n mssql-ag

NAME                     STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
instance-root-mssql1-0   Bound    pvc-92b9dbfd-0f3f-11ea-8d22-169104339902   5Gi        RWO            default        50m
instance-root-mssql2-0   Bound    pvc-944db4b7-0f3f-11ea-8d22-169104339902   5Gi        RWO            default        50m
instance-root-mssql3-0   Bound    pvc-9612e57a-0f3f-11ea-8d22-169104339902   5Gi        RWO            default        50m


In [63]:
%%bash
az disk list -g MC_RG_BDC_O9O9AKS_JAPANEAST

Name                                                                ResourceGroup                Location    Zones    Sku           OsType    SizeGb    ProvisioningState
------------------------------------------------------------------  ---------------------------  ----------  -------  ------------  --------  --------  -------------------
aks-nodepool1-15232104-0_OsDisk_1_233442b330ce48aeb803829051658ad3  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-1_OsDisk_1_1f8b8034fcce4ee2b97c024821b6d8bf  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-2_OsDisk_1_e0e8ca95bf704caeade77ebf95d47a82  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
kubernetes-dynamic-pvc-8f6f615f-0f43-11ea-8d22-169104339902         MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Standard_LRS            8         Succeeded
kubernetes

In [64]:
%%bash
kubectl get service -o wide -n mssql-ag


NAME           TYPE           CLUSTER-IP    EXTERNAL-IP      PORT(S)             AGE   SELECTOR
ag-primary     LoadBalancer   10.0.86.31    52.155.111.133   1433:32727/TCP      34m   role.ag.mssql.microsoft.com/mssql-ag=primary,type=sqlservr
ag-secondary   LoadBalancer   10.0.22.77    13.71.153.144    1433:32149/TCP      34m   role.ag.mssql.microsoft.com/mssql-ag=secondary,type=sqlservr
mssql-ag       ClusterIP      None          <none>           1433/TCP,5022/TCP   51m   ag-service.mssql.microsoft.com/mssql-ag=
mssql1         LoadBalancer   10.0.244.43   52.243.33.88     1433:30555/TCP      51m   name=mssql1,type=sqlservr
mssql2         LoadBalancer   10.0.217.1    13.73.22.201     1433:31521/TCP      51m   name=mssql2,type=sqlservr
mssql3         LoadBalancer   10.0.11.147   52.156.59.47     1433:32235/TCP      51m   name=mssql3,type=sqlservr


 # 6. AG確認（DMV）

In [65]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'select r.replica_server_name, rs.is_primary_replica, rs.synchronization_health_desc FROM sys.dm_hadr_database_replica_states rs JOIN sys.availability_replicas r ON r.group_id = rs.group_id AND r.replica_id = rs.replica_id'

replica_server_name                                                                                                                                                                                                                                              is_primary_replica synchronization_health_desc                                 
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ------------------ ------------------------------------------------------------
mssql1-0                                                                                                                                                                                                                                                                          0 HEALTHY                                           

In [66]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'select @@servername'


--------------------------------------------------------------------------------------------------------------------------------
mssql3-0                                                                                                                        

(1 rows affected)


In [67]:
%%bash
sqlcmd -S 13.71.153.144 -U sa -P $PASSWORD -Q 'select @@servername'



--------------------------------------------------------------------------------------------------------------------------------
mssql2-0                                                                                                                        

(1 rows affected)


 # 6. AG確認（SSMS）
 ![](image/2019-11-25-14-48-45.png)

 # 6. AG確認（K8s Dashboard）
 ![](image/2019-11-25-14-51-57.png)

In [70]:
%%bash
az aks browse -g rg_bdc --listen-address 0.0.0.0 -n o9o9aks


Process is interrupted.


 # 7.セカンダリから接続（ReadOnlyでないとエラーになる）

In [71]:
%%bash
sqlcmd -S 13.71.153.144 -U sa -P $PASSWORD -Q 'use agdb; select * from tab01'

Msg 978, Level 14, State 1, Server mssql2-0, Line 1
The target database ('agdb') is in an availability group and is currently accessible for connections when the application intent is set to read only. For more information about application intent, see SQL Server Books Online.


In [72]:
%%bash
sqlcmd -S 13.71.153.144 -U sa -P $PASSWORD -Q 'use agdb; select * from tab01' -K READONLY


Changed database context to 'agdb'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 AG on K8s!                                                                                                                                                                                                                                                      

(1 rows affected)


 # 8. AGの機能でフェールオーバー（エラーになる）

In [73]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'ALTER AVAILABILITY GROUP agdb FAILOVER'


Msg 15151, Level 16, State 1, Server mssql3-0, Line 1
Cannot alter the availability group 'agdb', because it does not exist or you do not have permission.


 # 9. K8sのJobでフェールオーバー

In [74]:
%%bash
# kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/4-1.failover_to_mssql1-0.yaml -n mssql-ag
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/4-2.failover_to_mssql2-0.yaml -n mssql-ag
# kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/4-3.failover_to_mssql3-0.yaml -n mssql-ag

serviceaccount/manual-failover created
role.rbac.authorization.k8s.io/manual-failover created
rolebinding.rbac.authorization.k8s.io/manual-failover created
job.batch/manual-failover created


In [75]:
%%bash
kubectl get jobs -n mssql-ag


NAME                      COMPLETIONS   DURATION   AGE
manual-failover           1/1           6s         17s
mssql-initialize-mssql1   1/1           16m        61m
mssql-initialize-mssql2   1/1           15m        61m
mssql-initialize-mssql3   1/1           2m59s      61m


 # 10. AG確認

In [76]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'select r.replica_server_name, rs.is_primary_replica, rs.synchronization_health_desc FROM sys.dm_hadr_database_replica_states rs JOIN sys.availability_replicas r ON r.group_id = rs.group_id AND r.replica_id = rs.replica_id'

replica_server_name                                                                                                                                                                                                                                              is_primary_replica synchronization_health_desc                                 
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ------------------ ------------------------------------------------------------
mssql1-0                                                                                                                                                                                                                                                                          0 HEALTHY                                           

In [77]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'select @@servername'


--------------------------------------------------------------------------------------------------------------------------------
mssql2-0                                                                                                                        

(1 rows affected)


In [78]:
%%bash
sqlcmd -S 13.71.153.144 -U sa -P $PASSWORD -Q 'select @@servername'



--------------------------------------------------------------------------------------------------------------------------------
mssql1-0                                                                                                                        

(1 rows affected)
